In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [14]:
# GroupBy machanics
df = DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'], 'key2' : ['one', 'two', 'one', 'two', 'one'], 'data1' : np.random.randn(5), 'data2' : np.random.randn(5)})
print df
grouped = df['data1'].groupby(df['key1'])
grouped.mean()
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means.unstack()

print df.groupby('key1')['data1'].mean()
print df['data1'].groupby(df['key1']).mean()
print df.groupby(['key1', 'key2'])[['data2']].mean()

states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

df.groupby(['key1', 'key2']).mean()
df.groupby(['key1', 'key2']).size()

      data1     data2 key1 key2
0 -0.638153  0.452041    a  one
1 -0.286540 -0.488708    a  two
2  0.836742 -0.246670    b  one
3 -0.447989  0.953819    b  two
4  0.494779 -1.090811    a  one
key1
a   -0.143305
b    0.194376
Name: data1, dtype: float64
key1
a   -0.143305
b    0.194376
Name: data1, dtype: float64
              data2
key1 key2          
a    one  -0.319385
     two  -0.488708
b    one  -0.246670
     two   0.953819


key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [15]:
# Iterating over groups
for name, group in df.groupby('key1'):
    print name
    print group
for (name1, name2), group in df.groupby(['key1', 'key2']):
    print name1, name2
    print group
pieces = dict(list(df.groupby('key1')))
pieces['b']
# group columns
grouped = df.groupby(df.dtypes, axis = 1)
dict(list(grouped))

a
      data1     data2 key1 key2
0 -0.638153  0.452041    a  one
1 -0.286540 -0.488708    a  two
4  0.494779 -1.090811    a  one
b
      data1     data2 key1 key2
2  0.836742 -0.246670    b  one
3 -0.447989  0.953819    b  two
a one
      data1     data2 key1 key2
0 -0.638153  0.452041    a  one
4  0.494779 -1.090811    a  one
a two
     data1     data2 key1 key2
1 -0.28654 -0.488708    a  two
b one
      data1    data2 key1 key2
2  0.836742 -0.24667    b  one
b two
      data1     data2 key1 key2
3 -0.447989  0.953819    b  two


{dtype('float64'):       data1     data2
 0 -0.638153  0.452041
 1 -0.286540 -0.488708
 2  0.836742 -0.246670
 3 -0.447989  0.953819
 4  0.494779 -1.090811, dtype('O'):   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one}

In [18]:
# Grouping with Dicts and Series
people = DataFrame(np.random.randn(5, 5), columns=['a', 'b', 'c', 'd', 'e'], index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.ix[2:3, ['b', 'c']] = np.nan
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f' : 'orange'}
people.groupby(mapping, axis = 1).sum()

map_series = Series(mapping)
people.groupby(mapping, axis = 1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [21]:
# Grouping with Functions
# Any function passed as a group key will be called once per index value, with the return values being used as the group names.
people.groupby(len).sum()
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).sum()

a         b         c         d         e
3 one -0.293854  1.305411  0.152154 -1.048127 -3.499468
  two  1.770823 -0.646974  1.486193 -0.517010 -0.194487
5 one -0.671636  0.520802  1.576364  0.227314 -1.691660
6 two  1.487637  0.975958 -1.540406 -1.229981  0.722622

In [24]:
# Grouping by index levels, keyword: 'level'
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'], [1, 3, 5, 1, 3]], names=['cty', 'tenor'])
hier_df = DataFrame(np.random.randn(4,5), columns = columns)
hier_df.groupby(level = 'cty', axis = 1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [39]:
# Data Aggregation: p277, count, sum, mean, median, std, car, min, max, prod, first, last
df.groupby('key1')['data1'].quantile(0.9)
# To use your own aggregation functions, pass any function agg method:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped = df.groupby('key1')
grouped.agg(peak_to_peak)
grouped.describe()

data1     data2
key1                          
a    count  3.000000  3.000000
     mean  -0.143305 -0.375826
     std    0.579889  0.777596
     min   -0.638153 -1.090811
     25%   -0.462347 -0.789760
     50%   -0.286540 -0.488708
     75%    0.104119 -0.018333
     max    0.494779  0.452041
b    count  2.000000  2.000000
     mean   0.194376  0.353575
     std    0.908443  0.848874
     min   -0.447989 -0.246670
     25%   -0.126807  0.053452
     50%    0.194376  0.353575
     75%    0.515559  0.653697
     max    0.836742  0.953819

In [52]:
tips = pd.read_csv('Dataset/WorkBook2.csv')
print tips
grouped = tips.groupby(['sex', 'smoker'])
grouped_pct = tips.groupby(['sex', 'smoker'])['tip_pct']
grouped_pct.agg('mean')
grouped_pct.agg(['mean', 'std', peak_to_peak])
# pass the name of the columns
grouped_pct.agg([('foo', 'mean'), ('bar', 'std')])

functions = ['count', 'mean', 'max']

result = grouped['tip_pct', 'total_bill'].agg(functions)
result['tip_pct']

ftuples = [('Durchshnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

# To apply different functions to columns: pass a dict to AGG that contains a mapping of column names and functions
grouped.agg({'tip': np.max, 'size': 'sum'})
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'], 'size': 'sum'})

# Returning Aggregated Data in "unindexes" Form
tips.groupby(['sex', 'smoker'], as_index = False).mean()

   total_bill   tip     sex smoker  day    time  size   tip_pct
0       16.99  1.01  Female     No  Sun  Dinner     2  0.059447
1       10.34  1.66    Male     No  Sun  Dinner     3  0.160542
2       21.01  3.50    Male     No  Sun  Dinner     3  0.166587
3       23.68  3.31    Male     No  Sun  Dinner     2  0.139780
4       24.59  3.61  Female     No  Sun  Dinner     4  0.146808
5       25.29  4.71    Male     No  Sun  Dinner     4  0.186240


,sex,smoker,total_bill,tip,size,tip_pct
0,Female,No,20.79,2.310,3,0.103127
1,Male,No,20.08,3.295,3,0.163287


In [72]:
# Group wise Operations and Transformations: TRANSFORM and APPLY methods
print df
k1_means = df.groupby('key1').mean().add_prefix('mean_')
pd.merge(df, k1_means, left_on = 'key1', right_index = True)

df.groupby('key1').transform(np.mean)
df.join(df.groupby('key1').transform(np.mean).add_prefix('mean_'))

def demean(arr):
    return arr - arr.mean()
demeaned = df.groupby('key1').transform(demean)
demeaned

      data1     data2 key1 key2
0 -0.638153  0.452041    a  one
1 -0.286540 -0.488708    a  two
2  0.836742 -0.246670    b  one
3 -0.447989  0.953819    b  two
4  0.494779 -1.090811    a  one


,data1,data2
0,-0.494849,0.827867
1,-0.143235,-0.112882
2,0.642366,-0.600244
3,-0.642366,0.600244
4,0.638084,-0.714985


In [87]:
# Apply: general split-apply-combine
def top(df, n = 5, column = 'tip_pct'):
    return df.sort_values(by = column)[-n:]
top(tips, n = 6)
tips.groupby('smoker').apply(top, n = 3, column = 'total_bill')

# unable the group key
tips.groupby('smoker', group_keys = False).apply(top, n = 3)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240


In [94]:
# Quantile and Bucker Analysis
frame = DataFrame({'data1': np.random.randn(1000), 'data2': np.random.randn(1000)})
factor = pd.cut(frame['data1'], 4)
grouped = frame['data2'].groupby(factor)
factor2 = pd.qcut(frame['data1'], 10, labels = False)  # do not show the buckets
grouped2 = frame['data2'].groupby(factor2)

def get_stats(group):
    return {'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean': group.mean()}
grouped.apply(get_stats).unstack()
grouped2.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.827487,-0.102403,-1.961988
1,100.0,2.905463,0.118277,-2.092737
2,100.0,2.362477,0.212673,-1.518431
3,100.0,2.355791,0.213510,-2.544113
4,100.0,2.515656,0.070305,-2.346273
5,100.0,1.901904,0.063601,-2.954573
6,100.0,1.978584,-0.043510,-2.370687
7,100.0,2.642357,0.011193,-1.648820
8,100.0,2.205764,-0.048150,-2.357473


In [99]:
# Filling missing values with group specific values
s = Series(np.random.rand(6))
s[::2] = np.nan
s.fillna(s.mean())

states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio         -1.717844
New York     -1.289478
Vermont       0.500000
Florida      -0.748194
Oregon        1.110094
Nevada       -1.000000
California   -0.668528
Idaho        -1.000000
dtype: float64

In [106]:
# Random sampling and permutation
suits = ['H', 'S', 'C', 'D']
card_val = (range(1, 11) + [10] * 3) * 4 
base_names = ['A'] + range(2, 11) + ['J', 'K', 'Q'] 
cards = []
for suit in ['H', 'S', 'C', 'D']: 
    cards.extend(str(num) + suit for num in base_names)
deck = Series(card_val, index=cards)

def draw(deck, n = 5):
    return deck.take(np.random.permutation(len(deck))[:n])
draw(deck)

get_suit = lambda card: card[-1]  # the last letter is suit
deck.groupby(get_suit).apply(draw, n = 2)
deck.groupby(get_suit, group_keys = False).apply(draw, n = 2)

4C    4
8C    8
2D    2
7D    7
3H    3
7H    7
3S    3
9S    9
dtype: int64

In [111]:
# Group weighted average and correlation
df = DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'], 'data': np.random.randn(8), 'weights': np.random.rand(8)})
print df
grouped = df.groupby('category')
getwavg = lambda g: np.average(g['data'], weights = g['weights'])
grouped.apply(getwavg)

  category      data   weights
0        a -1.114831  0.935013
1        a  0.377016  0.482599
2        a  2.047486  0.851598
3        a -0.066502  0.234393
4        b -1.050473  0.615976
5        b -0.429873  0.726883
6        b -0.303842  0.488955
7        b -1.390813  0.195146


category
a    0.346546
b   -0.680581
dtype: float64

In [112]:
# Group wise linear regression
import statsmodels.api as sm def 
regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars] 
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit() 
    return result.params
by_year.apply(regress, 'AAPL', ['SPX'])

In [128]:
# Pivot tables and cross tabulation, p293
print tips
tips.groupby(['sex', 'smoker']).mean()
tips.pivot_table(index=['sex', 'smoker'])

tips.pivot_table(index = ['sex', 'day'], columns = 'smoker', values = ['tip_pct', 'size'])
tips.pivot_table(index = ['sex', 'day'], columns = 'smoker', values = ['tip_pct', 'size'], margins = True)
tips.pivot_table(index = ['sex', 'smoker'], columns = 'day', values = 'tip_pct', margins = True, aggfunc = len, fill_value = 0)

   total_bill   tip     sex smoker  day    time  size   tip_pct
0       16.99  1.01  Female     No  Sun  Dinner     2  0.059447
1       10.34  1.66    Male     No  Sun  Dinner     3  0.160542
2       21.01  3.50    Male     No  Sun  Dinner     3  0.166587
3       23.68  3.31    Male     No  Sun  Dinner     2  0.139780
4       24.59  3.61  Female     No  Sun  Dinner     4  0.146808
5       25.29  4.71    Male     No  Sun  Dinner     4  0.186240


,day,Sun,All
sex,smoker,,
Female,No,2.0,2.0
Male,No,4.0,4.0
All,,6.0,6.0


In [131]:
# Cross tabulations (cresstab) is a special case of a pivot table that computes group frequencies
pd.crosstab([tips.sex, tips.day], tips.smoker, margins = True)



,smoker,No,All
sex,day,,
Female,Sun,2,2
Male,Sun,4,4
All,,6,6
